In [1]:
import pandas as pd

Ouvrire le fichier de pharmakon

In [2]:
PKON_file = r"C:\Users\Axel\Documents\Présentations\FragBank\Publication\Diagrammes\Sunburst\pkon23all-Chmochampv14nov.xlsx"

In [3]:
pkon_df = pd.read_excel(PKON_file)

Ne récupérer que les colonnes INCHIKEY; CLASSYFIRE_CLASS et CLASSYFIRE_SUBCLASS du fichier de pkon

In [4]:
nouveau_df = pkon_df[['INCHIKEY', 'CLASSYFIRE_CLASS', 'CLASSYFIRE_SUBCLASS']]

In [5]:
nouveau_df['INCHIKEY'] = nouveau_df['INCHIKEY'].astype(str)

C:\Users\Axel\AppData\Local\Temp\ipykernel_7632\662538400.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nouveau_df['INCHIKEY'] = nouveau_df['INCHIKEY'].astype(str)


ne récupérer que les short InChiKey

In [6]:
nouveau_df['INCHIKEY'] = nouveau_df['INCHIKEY'].apply(lambda x: x.split('-')[0] if x is not None else None)

C:\Users\Axel\AppData\Local\Temp\ipykernel_7632\1272673200.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nouveau_df['INCHIKEY'] = nouveau_df['INCHIKEY'].apply(lambda x: x.split('-')[0] if x is not None else None)


In [7]:
nouveau_df.head() # Affiche pour vérification

,INCHIKEY,CLASSYFIRE_CLASS,CLASSYFIRE_SUBCLASS
0,AAABMNXUOFPYQK,Pyranodioxins,NaN
1,AAAFZMYJJHWUPN,Organooxygen compounds,Carbohydrates and carbohydrate conjugates
2,AAAHTAUQXADPPM,Glycerophospholipids,Glycerophosphoglycerophosphoglycerols
3,AAAIKXYZKHSEGN,Glycerophospholipids,Glycerophosphoglycerophosphoglycerols
4,AAANZTDKTFGJLZ,Benzopyrans,1-benzopyrans


Ouvrire mon fichier où nous avons l'ensemble des InChiKey de nos bases de données

In [10]:
MSP_file = r"C:\Users\Axel\Documents\Présentations\FragBank\Publication\Diagrammes\FragBank_CLEAN\DB_Janvier_2024\CSV\ALL_DB_INCHIKEY.xlsx"

In [11]:
MSP_df = pd.read_excel(MSP_file)

In [12]:
MSP_df['INCHIKEY'] = MSP_df['INCHIKEY'].astype(str)

N'en récupérer égakement que les short InChiKey

In [13]:
MSP_df['INCHIKEY'] = MSP_df['INCHIKEY'].apply(lambda x: x.split('-')[0] if x is not None else None)

In [14]:
MSP_df.head() # Affichage pour vérification

,FILENAME,PREDICTED,INCHIKEY
0,MassBank,False,PUMXWMGECQIOGB
1,MassBank,False,CBUOBMSAFIYYEJ
2,MassBank,False,FNSQKFOXORBCCC
3,MassBank,False,HNMWDXUKPJZOQD
4,MassBank,False,GRFAWUZMBBQEMX


On complète nos InChiKey de nos bases de données avec les classyfire de PKON23

In [15]:
MSP_df = MSP_df.merge(nouveau_df[['INCHIKEY', 'CLASSYFIRE_CLASS', 'CLASSYFIRE_SUBCLASS']], on='INCHIKEY', how='left')

In [16]:
MSP_df = MSP_df.drop_duplicates(subset='INCHIKEY', keep='first')

In [17]:
MSP_df.head() # Affichage pour vérification

,FILENAME,PREDICTED,INCHIKEY,CLASSYFIRE_CLASS,CLASSYFIRE_SUBCLASS
0,MassBank,False,PUMXWMGECQIOGB,Prenol lipids,Sesquiterpenoids
2,MassBank,False,CBUOBMSAFIYYEJ,Prenol lipids,Monoterpenoids
4,MassBank,False,FNSQKFOXORBCCC,Tetracyclines,NaN
6,MassBank,False,HNMWDXUKPJZOQD,Isochromanequinones,NaN
11,MassBank,False,GRFAWUZMBBQEMX,NaN,NaN


On enregistre ce DF là

In [18]:
MSP_df.to_excel(r"C:\Users\Axel\Documents\Présentations\FragBank\Publication\Diagrammes\Sunburst\MSP_inchikeys_classyfire.xlsx", index=False)

In [19]:
null_df = MSP_df[MSP_df['CLASSYFIRE_CLASS'].isnull()] # NULL

In [20]:
null_df = null_df.drop_duplicates(subset='INCHIKEY', keep='first')

ici on enregistre le DF des short InChikey qui n'ont aucune information de classification taxonomique ==> c'est celui-ci que l'on utilise pour l'interrogation API de Classyfire

In [21]:
null_df.to_excel(r"C:\Users\Axel\Documents\Présentations\FragBank\Publication\Diagrammes\Sunburst\MSP_inchikeys_classyfire_null.xlsx", index=False)

In [25]:
print(len(null_df))

12670


Juste pour voir ce qu'on à réussi à récupérer de PKON 23

In [22]:
not_null_df = MSP_df[~MSP_df['CLASSYFIRE_CLASS'].isnull()] # NOT NULL

In [24]:
print(len(not_null_df))

18730


In [23]:
not_null_df.to_excel(r"C:\Users\Axel\Documents\Présentations\FragBank\Publication\Diagrammes\Sunburst\MSP_inchikeys_classyfire_not_null.xlsx", index=False)